# Web Hyperlink Matrix
*H* is a *n* x *n* **Hyperlink Matrix** where *n* is the number of webpages. If webpage *i* has *k* number of links to other webpages and webpage *i* links to webpage *j*, then the element in row *i* and column *j* is *1/k*. Otherwise, element in row *i* and column *j* is 0

\begin{align}
H_{ij} = \frac{1}{k}
\end{align}

I will use the following Hyperlink Matrix for the simple implementation version of PageRank algorithm

\begin{align}
H = 
 \begin{pmatrix}
  0 & 1 & 0 & 0 \\
  0 & 0 & 1 & 0 \\
  \frac{1}{2} & 0 & 0 & \frac{1}{2} \\
  0 & 0 & 0 & 0
 \end{pmatrix}
\end{align}

In [1]:
import numpy as np
H = np.matrix([[0, 1, 0, 0], 
               [0, 0, 1, 0],
               [1/2, 0, 0, 1/2],
               [0, 0, 0, 0]])
n = H.shape[1]

# Dangling Node Fix
**Dangling node** is a node that does not link to any other nodes. If a random surfer lands on a dangling node webpage, the surfer cannot go to any other webpages with clicking a link. 
There are many options to fix dangling node problem, but Google does not reveal which option it employs.

Since many literature suggests uniform probability distribution for the dangling node row, I will also adopt this method to model the Web Hyperlink Matrix

\begin{align}
S = H + dw
\end{align}

In [2]:
def fix_dangling_node(H):
    # Finds which rows have all elements zeros
    # d is a column vector that identifies dangling nodes
    d = ~(H.any(axis=1))
    # w is a uniform row vector
    w = np.full(H.shape[1], 1/H.shape[1])
    S = H + d*w
    return S

In [3]:
S = fix_dangling_node(H)
S

matrix([[ 0.  ,  1.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  1.  ,  0.  ],
        [ 0.5 ,  0.  ,  0.  ,  0.5 ],
        [ 0.25,  0.25,  0.25,  0.25]])

# Google Matrix

To model the overall behavior of a random web surfer, Google forms the matrix $G$
where $0 \le \alpha \lt 1$, $\mathbb{1}$ is the column vector of ones, and $v$ is a row probability distribution vector called *personalization vector*

\begin{align}
G = \alpha S + (1-\alpha)\mathbb{1}v 
\end{align}

**A damping factor** $\alpha$ in the Google matrix indicates that random web surfers move to a different webpage by some means other than selecting a link with probability $1-\alpha$

In [4]:
def form_google_matrix(S, v, alpha=0.85):
    one_vector = np.ones(S.shape[0])
    return alpha*S + (1-alpha)*one_vector*v

In [5]:
v = np.full(H.shape[1], 1/H.shape[1]) # Uniform vector
G = form_google_matrix(S, v)
G

matrix([[ 0.0375,  0.8875,  0.0375,  0.0375],
        [ 0.0375,  0.0375,  0.8875,  0.0375],
        [ 0.4625,  0.0375,  0.0375,  0.4625],
        [ 0.25  ,  0.25  ,  0.25  ,  0.25  ]])

# Computing PageRank scores

Since each element of $G_{ij}$ of $G$ lies between 0 and 1 ($0 \le G_{ij} \le 1$) and the sum of elements in each row of $G$ is 1, the Google Matrix $G$ is called *row stochastic matrix*. In addition, $\lambda = 1$ is not a repeated eigenvalue of $G$ and is greater in magnitude than any other eigenvalue of G. Therefore, the eigensystem $\pi G = \pi$ has a unique solution, where $\pi$is a **row probabability distribution vector**.

\begin{align}
\pi G = \pi
\end{align}

The $i$th entry of $\pi$ is the PageRank score for webpage $i$, and $\pi$ is called **PageRank vector**

## Power method

The oldest and easiest technique for approximating a dominant eigenvector of a matrix is **power method**. The power method converges when the dominant eigenvalue is not a repeated eigenvalue for most starting vectors. Since $\lambda = 1$ is the dominant eigenvalue of $G$ and $\pi$ is the dominant left eigenvector, *the power method applied to $G$ converges to the PageRank vector*

In [6]:
def power_method1(G):
    pi_new = np.full(H.shape[1], 1/H.shape[1])
    pi = np.zeros(H.shape[0])
    cnt = 0
    while not np.allclose(pi_new, pi):
        pi = pi_new
        pi_new = pi * G
        cnt += 1
    #print("Power method1 went through {} iteration".format(cnt))
    return pi_new    

'''
Here, we do not use pi = pi * G since multiplication of pi with G is expensive. 
Therefore, since S = H + dw and G = alpha * S + (1-alpha) 1 v, 
we can break down the equation for faster calculation
'''
def power_method2(H, v, alpha=0.85):
    d = ~(H.any(axis=1))
    w = np.full(H.shape[1], 1/H.shape[1])
    # start with pi = v
    pi = np.zeros(H.shape[0])
    pi_new = v
    cnt = 0
    while not np.allclose(pi_new, pi):
        pi = pi_new
        pi_new = alpha * pi * H + alpha * (pi*d) * w + (1-alpha) * v
        cnt += 1
    #print("Power method2 went through {} iteration".format(cnt))
    return pi_new


In [7]:
%timeit power_method1(G)

1000 loops, best of 3: 1.58 ms per loop


In [8]:
%timeit power_method2(H,v)

100 loops, best of 3: 2.21 ms per loop


In [9]:
v1 = np.array([1,0,0,0])
power_method2(H, v1)

matrix([[ 0.29698616,  0.28367298,  0.27235469,  0.14698616]])

In [10]:
power_method2(H, v, alpha=0.95)

matrix([[ 0.2115298 ,  0.26369286,  0.31324753,  0.2115298 ]])

In [11]:
power_method2(H, v1, alpha=0.95)

matrix([[ 0.23830397,  0.27111286,  0.30227919,  0.18830397]])

# Useful References
The majority of this jupyter notebook content is from [Google’s PageRank: The Math Behind the Search Engine](http://www.cems.uvm.edu/%7Etlakoba/AppliedUGMath/other_Google/Wills.pdf)
* [python - Finding which rows have all elements as zeros in a matrix with numpy - Stack Overflow](https://stackoverflow.com/questions/23726026/finding-which-rows-have-all-elements-as-zeros-in-a-matrix-with-numpy)